In [ ]:
include("_.jl")

In [ ]:
so_hg_ = Dict{String,Vector{String}}()

In [ ]:
hg = read_hgnc()

PR_ = string.(hg[hg[!, "locus_group"].=="protein-coding gene", "symbol"])

length(PR_)

## Ensembl

In [ ]:
en = TableAccess.read(joinpath(PAI, "ensembl", "human_to_mouse.tsv.gz"))

en[!, "Gene name"] = Gene.rename(string.(en[!, "Gene name"]))[1]

;

In [ ]:
hg_mo = Dict{String,String}()

for ro in eachrow(en)

    hg = ro["Gene name"]

    mo = ro["Mouse gene name"]

    if !(ismissing(hg) || ismissing(mo))

        hg_mo[hg] = mo

    end

end

println(length(hg_mo))

so_hg_["Ensembl"] = [pr for pr in PR_ if !haskey(hg_mo, pr)]

## Mouse Genome Informatics

In [ ]:
mg = TableAccess.read(joinpath(PAI, "HOM_MouseHumanSequence.rpt.txt.gz"))

se_ = mg[!, "Common Organism Name"] .== "human"

mg[se_, "Symbol"] .= Gene.rename(string.(mg[se_, "Symbol"]))[1]

;

In [ ]:
hg_mo = Dict{String,String}()

for da in groupby(mg, "DB Class Key")

    sy_ = da[!, "Symbol"]

    se_ = da[!, "Common Organism Name"] .== "human"

    for hg in sy_[se_], mo in sy_[.!se_]

        hg_mo[hg] = mo

    end

end

println(length(hg_mo))

so_hg_["MGI"] = [pr for pr in PR_ if !haskey(hg_mo, pr)]

## 2011 De Novo Origin of Human Protein-Coding Genes

In [ ]:
tr_ = readlines(joinpath(PAI, "De Novo Origin of Human Protein-Coding Genes", "genes.txt"))

so_hg_["2011 paper"] = Gene.rename(vcat(["CLLU1", "c22orf45", "DNAH10OS"], tr_))[1]

## 2019 Genes with human-specific features are primarily involved with brain, immune and metabolic evolution

In [ ]:
an_ge_ir = read_xlsx(
    joinpath(
        PAI,
        "Genes with human-specific features are primarily involved with brain, immune and metabolic evolution",
        "12859_2019_2886_MOESM2_ESM.xlsx",
    ),
    "HumanSpecific Genes",
)

so_hg_["2019 paper"] = Gene.rename(string.(an_ge_ir[!, "Gene Name"]))[1]

## Write

In [ ]:
hg_ = unique(vcat(values(so_hg_)...))

an_hg_so = DataFrame("HGNC Symbol" => hg_)

for (so, se_) in so_hg_

    an_hg_so[!, so] = [hg in se_ for hg in hg_]

end

an_hg_so[!, "Sum"] = vec(sum(Matrix(an_hg_so[!, 2:end]); dims = 2))

sort(an_hg_so, "Sum"; rev = true)

TableAccess.write(joinpath(PAO, "human_gene_by_soruce.tsv"), an_hg_so)